# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click !here goes the icon of the corresponding button in the gutter! button.
To debug a cell, press Alt+Shift+Enter, or click !here goes the icon of the corresponding button in the gutter! button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/jupyter-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [ ]:
#!pip install kagglehub
#!pip install seaborn
import kagglehub
import pandas as pd
import os
import numpy as np
import seaborn
from glob import glob
from datetime import datetime
from kaggle.api.kaggle_api_extended import KaggleApi
path = "data"

In [ ]:

def add_car_age(df):
    """
    Add a column calculating the difference between posting year and car year.
    Uses the posting_year column that was already created by clean_car_data.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing 'year' and 'posting_year' columns
    
    Returns:
    pandas.DataFrame: Original dataframe with new 'car_age' column
    """
    # Create a copy to avoid modifying the original
    df = df.copy()
    
    # Calculate the age of the car at posting time
    df['car_age'] = df['posting_year'] - df['year']
    
    return df

def process_and_clean_single_vehicle_file(file_path):
    """
    Load, clean, normalize a vehicle data file, calculate car age, 
    and save the cleaned DataFrame with a name including the earliest 
    and latest posting dates.
    
    Parameters:
    file_path (str): Path to the vehicle data CSV file to process
    
    Returns:
    None: The cleaned DataFrame is written to a new CSV file
    """
    print("Starting to process and clean the vehicle file...")

    # Extract the file name and determine posting_date if applicable
    file_name = os.path.basename(file_path)
    parts = file_name.replace('.csv', '').split('-')
    
    if len(parts) >= 3:
        year = parts[-1]
        month = parts[-2]
        default_posting_date = f"{year}-{month}-01"
    else:
        default_posting_date = None  # No default posting_date if the file doesn't follow the pattern
    
    # Load CSV into a DataFrame
    print(f"Reading file: {file_name}")
    df = pd.read_csv(file_path)
    
    # Add posting_date column if it doesn't exist
    if 'posting_date' not in df.columns:
        df['posting_date'] = default_posting_date
    
    # Clean and normalize the DataFrame
    print("Cleaning and normalizing data...")
    cleaned_df = clean_car_data(df)
    
    # Add car age
    print("Adding car age column...")
    cleaned_df = add_car_age(cleaned_df)
    
    # Convert posting_date to datetime for sorting and range determination
    cleaned_df['posting_date'] = pd.to_datetime(cleaned_df['posting_date'], errors='coerce')
    earliest_date = cleaned_df['posting_date'].min().strftime('%Y-%m-%d') if not cleaned_df['posting_date'].isna().all() else 'unknown'
    latest_date = cleaned_df['posting_date'].max().strftime('%Y-%m-%d') if not cleaned_df['posting_date'].isna().all() else 'unknown'
    
    # Construct the output file name
    output_file_name = f"cleaned-vehicles-{earliest_date}-{latest_date}.csv"
    output_path = os.path.join(os.path.dirname(file_path), output_file_name)
    
    # Write the cleaned DataFrame to the output file
    print(f"Writing cleaned data to: {output_file_name}")
    cleaned_df.to_csv(output_path, index=False)
    print("Processing and cleaning complete.")

def analyze_and_normalize_data(df):
    """
    Perform analysis and normalization of vehicle prices based on state-level factors.
    
    Parameters:
    df (pandas.DataFrame): Cleaned vehicle DataFrame
    
    Returns:
    pandas.DataFrame: DataFrame with normalized prices
    """
    print("Calculating state price factors...")
    state_factors = calculate_state_price_factors(df)
    print("State price factors calculated.")
    
    print("Adding normalized prices...")
    df_with_normalized = add_normalized_prices(df)
    print("Normalized prices added successfully.")
    
    return df_with_normalized
    



def clean_car_data(df):
    """
    Clean car listing data by:
    1. Removing rows with more than 2 missing values (ignoring specified columns)
    2. Filling single missing values with medians from similar cars
    
    Parameters:
    df (pandas.DataFrame): Raw car listing data
    
    Returns:
    pandas.DataFrame: Cleaned dataset
    """
    # Create a copy to avoid modifying original data
    cleaned_df = df.copy()
    
    # Extract year from posting_date string
    cleaned_df['posting_year'] = cleaned_df['posting_date'].str[:4].astype(float)
    
    # Columns to ignore in missing value calculations
    ignore_columns = ['long', 'lat', 'image_url', 'region', 'region_url', 'county', 'state']
    
    # Get columns to check for missing values (excluding ignored ones)
    columns_to_check = [col for col in cleaned_df.columns if col not in ignore_columns]
    
    # Remove rows with more than 2 missing values in the relevant columns
    rows_to_keep = cleaned_df[columns_to_check].isnull().sum(axis=1) <= 2
    cleaned_df = cleaned_df[rows_to_keep]
    
    # Function to fill missing values based on similar cars
    def fill_missing_value(row, column):
        if pd.isnull(row[column]):
            # Get median value for same make, model, year posted in same year
            similar_cars = cleaned_df[
                (cleaned_df['year'] == row['year']) &
                (cleaned_df['manufacturer'] == row['manufacturer']) &
                (cleaned_df['model'] == row['model']) &
                (cleaned_df['posting_year'] == row['posting_year']) &
                (~pd.isnull(cleaned_df[column]))
            ]
            
            if len(similar_cars) >= 3:  # If we have enough similar cars
                return similar_cars[column].median()
            
            # If not enough similar cars, broaden criteria (ignore model)
            similar_cars = cleaned_df[
                (cleaned_df['year'] == row['year']) &
                (cleaned_df['manufacturer'] == row['manufacturer']) &
                (cleaned_df['posting_year'] == row['posting_year']) &
                (~pd.isnull(cleaned_df[column]))
            ]
            
            if len(similar_cars) >= 3:
                return similar_cars[column].median()
            
            # If still not enough, use global median for that year
            return cleaned_df[
                (cleaned_df['year'] == row['year']) &
                (~pd.isnull(cleaned_df[column]))
            ][column].median()
    
    # Fill single missing values for numeric columns
    numeric_columns = ['odometer', 'price']  # Add other numeric columns as needed
    for column in numeric_columns:
        mask = cleaned_df[column].isnull()
        cleaned_df.loc[mask, column] = cleaned_df[mask].apply(
            lambda row: fill_missing_value(row, column), axis=1
        )
    
    return cleaned_df

def process_all_vehicle_files(folder_path):
    """
    Process and clean all vehicle files in a given folder.
    
    Parameters:
    folder_path (str): Path to the folder containing vehicle data CSV files
    
    Returns:
    None: Processed files are saved with new names in the same folder
    """
    print(f"Processing all vehicle files in folder: {folder_path}")
    
    # Get a list of all CSV files in the folder starting with 'vehicles'
    file_paths = glob(os.path.join(folder_path, "vehicles*.csv"))
    
    if not file_paths:
        print("No files found in the folder matching the pattern 'vehicles*.csv'.")
        return
    
    for idx, file_path in enumerate(file_paths, 1):
        print(f"Processing file {idx}/{len(file_paths)}: {os.path.basename(file_path)}")
        try:
            process_and_clean_single_vehicle_file(file_path)
            print(f"File {idx} processed successfully.")
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")
    
    print("All files in the folder have been processed.")

In [ ]:
#process_all_vehicle_files(path)


In [ ]:
def process_csv_files(input_folder):
    """
    Processes all cleaned CSV files in the input folder, calculates state price factors,
    and outputs the aggregated result to 'state_price_factors.csv'.

    Parameters:
    input_folder (str): Path to the folder containing the cleaned CSV files.
    
    Returns:
    pandas.DataFrame: Aggregated state price factors.
    """
    
    def calculate_state_price_factors(df):
        """
        Calculate how much each state's prices differ from the national average,
        with outlier removal and minimum listing requirements.

        Parameters:
        df (pandas.DataFrame): Cleaned car listing DataFrame

        Returns:
        pandas.DataFrame: State price factors showing percentage difference from national average
        """
        
        # Create copy for analysis
        df_analysis = df.copy()
        
        # Remove national outliers first
        Q1 = df_analysis['price'].quantile(0.25)
        Q3 = df_analysis['price'].quantile(0.75)
        IQR = Q3 - Q1
        price_lower_bound = Q1 - 1.5 * IQR
        price_upper_bound = Q3 + 1.5 * IQR
        
        df_analysis = df_analysis[
            (df_analysis['price'] >= price_lower_bound) &
            (df_analysis['price'] <= price_upper_bound)
        ]
        
        # Calculate national average after removing outliers
        national_avg = df_analysis['price'].mean()
        
        # Function to remove outliers for a specific state
        def get_state_mean_without_outliers(state_data):
            if len(state_data) < 100:  # Minimum listings requirement
                return None
                
            # Remove state-level outliers
            Q1 = state_data['price'].quantile(0.25)
            Q3 = state_data['price'].quantile(0.75)
            IQR = Q3 - Q1
            lower = Q1 - 1.5 * IQR
            upper = Q3 + 1.5 * IQR
            
            clean_state_data = state_data[
                (state_data['price'] >= lower) &
                (state_data['price'] <= upper)
            ]
            
            if len(clean_state_data) < 50:  # Minimum cleaned listings requirement
                return None
                
            return clean_state_data['price'].mean()
        
        # Calculate state-level statistics
        state_stats = []
        
        for state in df_analysis['state'].unique():
            state_data = df_analysis[df_analysis['state'] == state]
            state_mean = get_state_mean_without_outliers(state_data)
            
            if state_mean is not None:
                price_factor = (state_mean / national_avg - 1) * 100
                
                # Skip states with unrealistic price factors
                if abs(price_factor) <= 50:  # Maximum allowed deviation
                    state_stats.append({
                        'state': state,
                        'avg_price': state_mean,
                        'price_factor': price_factor,
                        'listing_count': len(state_data)
                    })
        
        # Convert to DataFrame
        state_stats = pd.DataFrame(state_stats)
        
        # Sort by price factor
        state_stats = state_stats.sort_values('price_factor', ascending=False)
        
        # Round numbers for readability
        state_stats['avg_price'] = state_stats['avg_price'].round(2)
        state_stats['price_factor'] = state_stats['price_factor'].round(2)
        
        return state_stats

    # Find all cleaned CSV files
    csv_files = [os.path.join(input_folder, f) for f in os.listdir(input_folder) 
                 if f.startswith('cleaned-') and f.endswith('.csv')]
    
    # Aggregate data from all files
    aggregated_df = pd.DataFrame()
    for file in csv_files:
        print(f"starting work on {file}") 
        df = pd.read_csv(file)
        aggregated_df = pd.concat([aggregated_df, df], ignore_index=True)
    
    # Calculate state price factors
    state_price_factors = calculate_state_price_factors(aggregated_df)
    
    # Save the result to CSV
    output_file = os.path.join(input_folder, 'state_price_factors.csv')
    state_price_factors.to_csv(output_file, index=False)
    
    return state_price_factors

def adjust_price_for_location(price, state, state_factors):
    """
    Adjust a car's price to account for state-level price differences.
    Returns original price if state adjustment isn't available.
    
    Parameters:
    price (float): Original price
    state (str): State where the car is listed
    state_factors (pandas.DataFrame): DataFrame with state price factors
    
    Returns:
    float: Nationally adjusted price
    """
    try:
        # Get the state's price factor
        state_factor = state_factors.loc[state_factors['state'] == state, 'price_factor'].iloc[0]
        
        # Adjust price by removing state factor
        adjusted_price = price / (1 + state_factor/100)
        
        return round(adjusted_price, 2)
    except:
        # Return original price if state adjustment isn't available
        return price

def add_normalized_prices(df):
    """
    Add a normalized_price column to the DataFrame that adjusts for state-level price differences.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing car listings
    
    Returns:
    pandas.DataFrame: Original DataFrame with new normalized_price column
    """
    # Create a copy to avoid modifying the original
    df_with_normalized = df.copy()
    
    # Calculate state factors
    state_factors = calculate_state_price_factors(df)
    
    # Create normalized_price column using vectorized operations
    df_with_normalized['state_price_factor'] = df_with_normalized['state'].map(
        state_factors.set_index('state')['price_factor'].fillna(0)
    )
    
    df_with_normalized['normalized_price'] = df_with_normalized.apply(
        lambda row: adjust_price_for_location(row['price'], row['state'], state_factors),
        axis=1
    )
    
    return df_with_normalized


def visualize_state_prices(filename='state_price_factors.csv'):
    """
    Read and visualize state price factors data.
    
    Parameters:
    filename (str): Path to the state price factors CSV file
    """
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    try:
        # Read the CSV file
        df = pd.read_csv(filename)
        
        # Sort by price factor
        df = df.sort_values('price_factor')
        
        # Create figure with two subplots
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), height_ratios=[2, 1])
        
        # Bar plot
        sns.barplot(data=df, x='state', y='price_factor', ax=ax1)
        ax1.set_title('State Price Differences from National Average')
        ax1.set_xlabel('State')
        ax1.set_ylabel('Price Difference (%)')
        ax1.tick_params(axis='x', rotation=45)
        
        # Add horizontal line at 0
        ax1.axhline(y=0, color='black', linestyle='-', alpha=0.2)
        
        # Text summary
        summary_text = (
            f"Summary Statistics:\n"
            f"Number of states: {len(df)}\n"
            f"Price factor range: {df['price_factor'].min():.1f}% to {df['price_factor'].max():.1f}%\n"
            f"Total listings analyzed: {df['listing_count'].sum():,}\n\n"
            f"Top 3 most expensive states:\n"
            + "\n".join(f"{state}: +{factor:.1f}%" 
                       for state, factor in df.nlargest(3, 'price_factor')[['state', 'price_factor']].values)
            + "\n\nTop 3 least expensive states:\n"
            + "\n".join(f"{state}: {factor:.1f}%" 
                       for state, factor in df.nsmallest(3, 'price_factor')[['state', 'price_factor']].values)
        )
        
        ax2.text(0.05, 0.95, summary_text, 
                transform=ax2.transAxes, 
                verticalalignment='top',
                fontfamily='monospace')
        ax2.axis('off')
        
        # Adjust layout and display
        plt.tight_layout()
        plt.show()
        
        print("\nDetailed Statistics:")
        print(df.sort_values('price_factor', ascending=False)\
              [['state', 'price_factor', 'avg_price', 'listing_count']]\
              .to_string(index=False))
        
    except FileNotFoundError:
        print(f"Error: Could not find file {filename}")
    except Exception as e:
        print(f"Error visualizing data: {str(e)}")

In [ ]:
result = process_csv_files("data/")
print(result)
visualize_state_prices('data/state_price_factors.csv')

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def get_oldest_and_newest_posts(folder_path):
    """
    Get the oldest and newest posts based on file names containing date ranges.
    File format expected: cleaned-vehicles-YYYY-MM-DD-YYYY-MM-DD.csv
    where the first date is the oldest post and second date is the newest post in that file.
    
    Parameters:
    folder_path (str): Path to the folder containing the files.
    
    Returns:
    dict: A dictionary containing the oldest and newest posts with their corresponding files.
    """
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"The folder '{folder_path}' does not exist.")
    
    # Get all files in the folder
    files = [f for f in os.listdir(folder_path) if f.startswith('cleaned-vehicles-') and f.endswith('.csv')]
    
    if not files:
        return {"oldest": None, "newest": None}
    
    # Extract dates from file names
    posts = []
    for file in files:
        try:
            # Remove prefix and suffix
            date_part = file.replace('cleaned-vehicles-', '').replace('.csv', '')
            
            # Split into two dates
            dates = date_part.split('-')
            
            # Reconstruct the dates (assuming YYYY-MM-DD-YYYY-MM-DD format)
            first_date_str = f"{dates[0]}-{dates[1]}-{dates[2]}"
            second_date_str = f"{dates[3]}-{dates[4]}-{dates[5]}"
            
            # Convert to datetime objects
            first_date = datetime.strptime(first_date_str, "%Y-%m-%d")
            last_date = datetime.strptime(second_date_str, "%Y-%m-%d")
            
            posts.append({
                "file": file,
                "first_date": first_date,
                "last_date": last_date
            })
        except (IndexError, ValueError) as e:
            print(f"Skipping file with invalid format: {file}. Error: {str(e)}")
            continue
    
    if not posts:
        return {"oldest": None, "newest": None}
    
    # Find the file with the earliest first date and the latest last date
    oldest_post = min(posts, key=lambda x: x["first_date"])
    newest_post = max(posts, key=lambda x: x["last_date"])
    
    return {
        "oldest": {
            "file": oldest_post["file"],
            "first_date": oldest_post["first_date"].strftime("%Y-%m-%d"),
        },
        "newest": {
            "file": newest_post["file"],
            "last_date": newest_post["last_date"].strftime("%Y-%m-%d"),
        }
    }

def normalize_date(date_str, oldest_date_str, newest_date_str):
    """
    Normalize a date to a value between -1 and 1, where:
    - oldest_date maps to -1
    - newest_date maps to 1
    
    Parameters:
    date_str (str): Date to normalize in YYYY-MM-DD format
    oldest_date_str (str): Reference start date in YYYY-MM-DD format
    newest_date_str (str): Reference end date in YYYY-MM-DD format
    
    Returns:
    float: Normalized value between -1 and 1
    """
    date = datetime.strptime(date_str, "%Y-%m-%d")
    oldest_date = datetime.strptime(oldest_date_str, "%Y-%m-%d")
    newest_date = datetime.strptime(newest_date_str, "%Y-%m-%d")
    
    # Calculate the total range in days
    total_range = (newest_date - oldest_date).days
    if total_range == 0:
        return 0  # If oldest and newest are the same date
    
    # Calculate where our date falls within that range
    days_from_start = (date - oldest_date).days
    
    # Normalize to [-1, 1] range
    normalized_value = (2 * days_from_start / total_range) - 1
    
    return normalized_value

def get_normalized_date_range(folder_path):
    """
    Get the date range and provide a normalization function for dates within that range.
    
    Parameters:
    folder_path (str): Path to the folder containing the files
    
    Returns:
    tuple: (date_range_dict, normalizer_function)
    """
    date_range = get_oldest_and_newest_posts(folder_path)
    
    if date_range["oldest"] is None or date_range["newest"] is None:
        return date_range, None
    
    oldest_date = date_range["oldest"]["first_date"]
    newest_date = date_range["newest"]["last_date"]
    
    # Create a partial function with the date range pre-set
    from functools import partial
    normalizer = partial(normalize_date, 
                        oldest_date_str=oldest_date,
                        newest_date_str=newest_date)
    
    return date_range, normalizer

def analyze_feature_impact_across_files(folder_path, normalizer=None, correlation_series=None):
    """
    Analyzes the impact of features on state-normalized price across multiple files,
    or visualizes existing correlation series.
    
    Parameters:
    folder_path (str): Path to folder containing the vehicle data CSV files
    normalizer (DataNormalizer, optional): Pre-fitted DataNormalizer instance
    correlation_series (pd.Series, optional): Existing correlation series to visualize
    
    Returns:
    tuple: (correlation_series, normalizer) - The correlation results and the normalizer instance used
    """
    
    
    def visualize_correlations(correlation_series):
        """Helper function to visualize correlation series"""
        # Sort correlations from smallest to largest
        correlation_series = correlation_series.sort_values()
        
        plt.figure(figsize=(12, 6))
        sns.barplot(x=correlation_series.index, y=correlation_series.values)
        plt.xticks(rotation=45, ha='right')
        plt.title("Feature Correlation with State-Normalized Price")
        plt.ylabel("Correlation Coefficient")
        plt.xlabel("Feature")
        plt.axhline(y=0, color='black', linestyle='-', alpha=0.2)
        
        # Calculate dynamic offset based on data range
        y_range = correlation_series.max() - correlation_series.min()
        offset = y_range * 0.01  # 1% of total range
        
        # Add correlation values on top of bars
        for i, v in enumerate(correlation_series):
            plt.text(i, v + (offset if v >= 0 else -offset),
                    f'{v:.3f}',
                    ha='center',
                    va='bottom' if v >= 0 else 'top',
                    rotation=90)
        
        plt.tight_layout()
        plt.show()
    
    # Print correlations
    print("\nFeature correlations with state-normalized price:")
    for feature, corr in correlation_series.items():
        print(f"{feature:15} : {corr:>8.3f}")
    
    # If correlation_series is provided, just visualize it and return
    if correlation_series is not None:
        visualize_correlations(correlation_series)
        return correlation_series, normalizer
    
    # Load state price factors first
    state_factors_path = os.path.join(folder_path, 'state_price_factors.csv')
    if not os.path.exists(state_factors_path):
        print("State price factors file not found. Please run process_csv_files first.")
        return None, normalizer
    
    state_factors = pd.read_csv(state_factors_path)
    
    # Get list of cleaned vehicle files
    file_paths = glob(os.path.join(folder_path, "cleaned-vehicles-*.csv"))
    
    if not file_paths:
        print("No files found matching the pattern 'cleaned-vehicles-*.csv'")
        return None, normalizer
    
    # Initialize correlation tracking
    correlations_list = []
    
    # Columns to completely exclude from analysis
    exclude_columns = [
        'id', 'url', 'region', 'region_url', 'price', 'lat', 'long',
        'posting_date', 'image_url', 'county', 'description', 'VIN',
        'state', 'normalized_price', 'state_normalized_price', 'Unnamed: 0'  # Added Unnamed: 0
    ]
    
    def adjust_price_for_state(df):
        """Helper function to adjust prices based on state factors"""
        df = df.copy()
        state_price_map = state_factors.set_index('state')['price_factor'].to_dict()
        df['state_normalized_price'] = df.apply(
            lambda row: row['price'] / (1 + state_price_map.get(row['state'], 0)/100), 
            axis=1
        )
        return df
    
    # Process each file
    for file_path in file_paths:
        print(f"Processing {os.path.basename(file_path)}...")
        
        try:
            # Read the file without index column
            df = pd.read_csv(file_path)
            
            # Normalize prices by state first
            df = adjust_price_for_state(df)
            
            # Drop excluded columns
            analysis_df = df.drop(columns=[col for col in exclude_columns if col in df.columns])
            
            # Calculate correlations for each feature
            for column in analysis_df.columns:
                if column != 'state_normalized_price':
                    # Skip non-numeric columns and convert categorical if possible
                    if not np.issubdtype(analysis_df[column].dtype, np.number):
                        if analysis_df[column].dtype == 'object':
                            # Try to convert categorical to numeric using label encoding
                            try:
                                numeric_values = pd.Categorical(analysis_df[column]).codes
                                if len(set(numeric_values)) > 1:  # Only if we have multiple categories
                                    corr = np.corrcoef(
                                        numeric_values, 
                                        df['state_normalized_price'].values
                                    )[0,1]
                                    if not np.isnan(corr):
                                        correlations_list.append({'feature': column, 'correlation': corr})
                            except:
                                continue
                    else:
                        # For numeric columns, calculate correlation directly
                        valid_data = analysis_df[[column]].join(df['state_normalized_price']).dropna()
                        if len(valid_data) > 100:  # Only if we have enough data points
                            corr = valid_data[column].corr(valid_data['state_normalized_price'])
                            if not np.isnan(corr):
                                correlations_list.append({'feature': column, 'correlation': corr})
        
        except Exception as e:
            print(f"Error processing {file_path}: {str(e)}")
            continue
    
    if not correlations_list:
        print("No correlations were calculated. Check if the data contains numeric columns.")
        return None, normalizer
    
    # Average correlations across files
    correlation_df = pd.DataFrame(correlations_list)
    correlation_series = correlation_df.groupby('feature')['correlation'].mean()
    
    # Sort by correlation value (ascending)
    correlation_series = correlation_series.sort_values()
    
    # Visualize the correlations
    visualize_correlations(correlation_series)
    
    return correlation_series, normalizer

class DataNormalizer:
    def __init__(self):
        # Ordinal mappings
        self.size_order = {
            'sub-compact': 0,
            'compact': 1,
            'mid-size': 2,
            'full-size': 3
        }
        
        self.cylinder_order = {
            'other': 0,
            '3 cylinders': 3,
            '4 cylinders': 4,
            '5 cylinders': 5,
            '6 cylinders': 6,
            '8 cylinders': 8,
            '10 cylinders': 10,
            '12 cylinders': 12
        }
        
        self.condition_order = {
            'salvage': 0,
            'fair': 1,
            'good': 2,
            'excellent': 3,
            'like new': 4,
            'new': 5
        }
        
        # Reverse mappings for denormalization
        self.size_reverse = {v: k for k, v in self.size_order.items()}
        self.cylinder_reverse = {v: k for k, v in self.cylinder_order.items()}
        self.condition_reverse = {v: k for k, v in self.condition_order.items()}
        
        # Storage for normalization parameters
        self.numeric_params = {}
        self.categorical_mappings = {}
        
        # Storage for imputation mappings
        self.model_cylinders_map = {}
        self.manufacturer_cylinders_map = {}
        self.manufacturer_model_map = {}
        self.most_common_cylinders = None
        self.most_common_model = None
        
        # Define column types
        self.numeric_cols = ['price', 'year', 'odometer']
        self.categorical_cols = [
            'fuel', 'title_status', 'transmission', 
            'drive', 'type', 'state', 'county'
        ]
        self.encode_only_cols = ['paint_color', 'region', 'manufacturer', 'model']

    def fit(self, file_paths):
        """
        Calculate normalization parameters and imputation mappings from multiple files.
        
        Parameters:
        file_paths (list): List of paths to CSV files
        """
        print("Calculating normalization parameters and imputation mappings...")
        
        # Initialize parameters for numeric columns
        numeric_mins = {col: float('inf') for col in self.numeric_cols}
        numeric_maxs = {col: float('-inf') for col in self.numeric_cols}
        
        # Initialize sets for categorical values
        categorical_values = {col: set() for col in self.categorical_cols}
        encode_only_values = {col: set() for col in self.encode_only_cols}
        
        # Initialize counters for imputation
        model_cylinders_count = {}      # {model: {cylinders: count}}
        manufacturer_cylinders_count = {}  # {manufacturer: {cylinders: count}}
        manufacturer_model_count = {}    # {manufacturer: {model: count}}
        cylinders_count = {}            # {cylinders: count}
        model_count = {}                # {model: count}
        
        # Process each file
        for file_path in file_paths:
            print(f"Processing {os.path.basename(file_path)}...")
            
            # Read file in chunks to save memory
            chunk_size = 10000
            for chunk in pd.read_csv(file_path, chunksize=chunk_size):
                # Update numeric min/max
                for col in self.numeric_cols:
                    if col in chunk.columns:
                        chunk_min = chunk[col].dropna().min()
                        chunk_max = chunk[col].dropna().max()
                        if not pd.isna(chunk_min) and not pd.isna(chunk_max):
                            numeric_mins[col] = min(numeric_mins[col], chunk_min)
                            numeric_maxs[col] = max(numeric_maxs[col], chunk_max)
                
                # Update categorical values
                for col in self.categorical_cols:
                    if col in chunk.columns:
                        categorical_values[col].update(chunk[col].dropna().unique())
                
                # Update encode-only values
                for col in self.encode_only_cols:
                    if col in chunk.columns:
                        encode_only_values[col].update(chunk[col].dropna().unique())
                
                # Count relationships for imputation
                valid_rows = chunk[['manufacturer', 'model', 'cylinders']].dropna(subset=['manufacturer'])
                
                for _, row in valid_rows.iterrows():
                    mfr = row['manufacturer']
                    model = row['model']
                    cyls = row['cylinders']
                    
                    # Count cylinders by model
                    if pd.notna(model) and pd.notna(cyls):
                        if model not in model_cylinders_count:
                            model_cylinders_count[model] = {}
                        model_cylinders_count[model][cyls] = model_cylinders_count[model].get(cyls, 0) + 1
                    
                    # Count cylinders by manufacturer
                    if pd.notna(cyls):
                        if mfr not in manufacturer_cylinders_count:
                            manufacturer_cylinders_count[mfr] = {}
                        manufacturer_cylinders_count[mfr][cyls] = manufacturer_cylinders_count[mfr].get(cyls, 0) + 1
                        cylinders_count[cyls] = cylinders_count.get(cyls, 0) + 1
                    
                    # Count models by manufacturer
                    if pd.notna(model):
                        if mfr not in manufacturer_model_count:
                            manufacturer_model_count[mfr] = {}
                        manufacturer_model_count[mfr][model] = manufacturer_model_count[mfr].get(model, 0) + 1
                        model_count[model] = model_count.get(model, 0) + 1
        
        # Store numeric parameters
        for col in self.numeric_cols:
            if numeric_mins[col] < numeric_maxs[col]:
                self.numeric_params[col] = {
                    'min': numeric_mins[col],
                    'max': numeric_maxs[col]
                }
        
        # Create categorical mappings
        for col in self.categorical_cols:
            if categorical_values[col]:
                sorted_values = sorted(categorical_values[col])
                mapping = {val: idx for idx, val in enumerate(sorted_values)}
                reverse_mapping = {idx: val for val, idx in mapping.items()}
                self.categorical_mappings[col] = {
                    'mapping': mapping,
                    'reverse_mapping': reverse_mapping,
                    'max': len(mapping) - 1
                }
        
        # Create encode-only mappings
        for col in self.encode_only_cols:
            if encode_only_values[col]:
                sorted_values = sorted(encode_only_values[col])
                mapping = {val: idx for idx, val in enumerate(sorted_values)}
                reverse_mapping = {idx: val for val, idx in mapping.items()}
                self.categorical_mappings[col] = {
                    'mapping': mapping,
                    'reverse_mapping': reverse_mapping
                }
        
        # Create imputation mappings
        # Most common cylinders by model
        for model, cyls_dict in model_cylinders_count.items():
            self.model_cylinders_map[model] = max(cyls_dict.items(), key=lambda x: x[1])[0]
        
        # Most common cylinders by manufacturer
        for mfr, cyls_dict in manufacturer_cylinders_count.items():
            self.manufacturer_cylinders_map[mfr] = max(cyls_dict.items(), key=lambda x: x[1])[0]
        
        # Most common model by manufacturer
        for mfr, model_dict in manufacturer_model_count.items():
            self.manufacturer_model_map[mfr] = max(model_dict.items(), key=lambda x: x[1])[0]
        
        # Global most common values
        if cylinders_count:
            self.most_common_cylinders = max(cylinders_count.items(), key=lambda x: x[1])[0]
        if model_count:
            self.most_common_model = max(model_count.items(), key=lambda x: x[1])[0]
        
        print("Normalization parameters and imputation mappings calculated.")
        
        # Print encoding ranges for encode-only columns
        print("\nEncoding ranges:")
        for col in self.encode_only_cols:
            if col in self.categorical_mappings:
                n_values = len(self.categorical_mappings[col]['mapping'])
                print(f"{col}: 0 to {n_values - 1} ({n_values} unique values)")

    def normalize(self, df):
        """
        Normalize the DataFrame using stored parameters, with smart imputation for missing values.
        
        Parameters:
        df (pandas.DataFrame): Input DataFrame
        
        Returns:
        pandas.DataFrame: Normalized DataFrame with imputed missing values
        """
        if not self.numeric_params and not self.categorical_mappings:
            raise ValueError("Normalizer has not been fitted. Call fit() first.")
            
        df_norm = df.copy()
        
        # Impute missing cylinders
        mask = df_norm['cylinders'].isna()
        if mask.any():
            for idx in df_norm[mask].index:
                model = df_norm.loc[idx, 'model']
                manufacturer = df_norm.loc[idx, 'manufacturer']
                
                # Try to fill based on model
                if pd.notna(model) and model in self.model_cylinders_map:
                    df_norm.loc[idx, 'cylinders'] = self.model_cylinders_map[model]
                # Try to fill based on manufacturer
                elif pd.notna(manufacturer) and manufacturer in self.manufacturer_cylinders_map:
                    df_norm.loc[idx, 'cylinders'] = self.manufacturer_cylinders_map[manufacturer]
                # Use global most common
                elif self.most_common_cylinders is not None:
                    df_norm.loc[idx, 'cylinders'] = self.most_common_cylinders
        
        # Impute missing models
        mask = df_norm['model'].isna()
        if mask.any():
            for idx in df_norm[mask].index:
                manufacturer = df_norm.loc[idx, 'manufacturer']
                
                # Try to fill based on manufacturer
                if pd.notna(manufacturer) and manufacturer in self.manufacturer_model_map:
                    df_norm.loc[idx, 'model'] = self.manufacturer_model_map[manufacturer]
                # Use global most common
                elif self.most_common_model is not None:
                    df_norm.loc[idx, 'model'] = self.most_common_model
        
        # Normalize numeric columns
        for col in self.numeric_cols:
            if col in df_norm.columns and col in self.numeric_params:
                params = self.numeric_params[col]
                df_norm[col] = (df_norm[col] - params['min']) / (params['max'] - params['min'])
        
        # Normalize ordinal columns
        # Size
        if 'size' in df_norm.columns:
            df_norm['size'] = df_norm['size'].map(self.size_order)
            size_max = max(self.size_order.values())
            df_norm['size'] = df_norm['size'] / size_max
        
        # Cylinders
        if 'cylinders' in df_norm.columns:
            df_norm['cylinders'] = df_norm['cylinders'].map(self.cylinder_order)
            cylinder_max = max(self.cylinder_order.values())
            df_norm['cylinders'] = df_norm['cylinders'] / cylinder_max
        
        # Condition
        if 'condition' in df_norm.columns:
            df_norm['condition'] = df_norm['condition'].map(self.condition_order)
            condition_max = max(self.condition_order.values())
            df_norm['condition'] = df_norm['condition'] / condition_max
        
        # Normalize categorical columns to 0-1
        for col in self.categorical_cols:
            if col in df_norm.columns and col in self.categorical_mappings:
                mapping = self.categorical_mappings[col]['mapping']
                max_val = self.categorical_mappings[col]['max']
                df_norm[col] = df_norm[col].map(mapping) / max_val
        
        # Handle encode-only columns (simple integer encoding)
        for col in self.encode_only_cols:
            if col in df_norm.columns and col in self.categorical_mappings:
                mapping = self.categorical_mappings[col]['mapping']
                df_norm[col] = df_norm[col].map(mapping)
        
        return df_norm

    def denormalize(self, df_norm):
        """
        Denormalize the DataFrame using stored parameters.
        
        Parameters:
        df_norm (pandas.DataFrame): Normalized DataFrame
        
        Returns:
        pandas.DataFrame: Denormalized DataFrame with original scale and categories
        """
        if not self.numeric_params and not self.categorical_mappings:
            raise ValueError("Normalizer has not been fitted. Call fit() first.")
            
        df_denorm = df_norm.copy()
        
        # Denormalize numeric columns
        for col in self.numeric_cols:
            if col in df_denorm.columns and col in self.numeric_params:
                params = self.numeric_params[col]
                df_denorm[col] = df_denorm[col] * (params['max'] - params['min']) + params['min']
        
        # Denormalize ordinal columns
        # Size
        if 'size' in df_denorm.columns:
            size_max = max(self.size_order.values())
            df_denorm['size'] = (df_denorm['size'] * size_max).round()
            df_denorm['size'] = df_denorm['size'].map(self.size_reverse)
        
        # Cylinders
        if 'cylinders' in df_denorm.columns:
            cylinder_max = max(self.cylinder_order.values())
            df_denorm['cylinders'] = (df_denorm['cylinders'] * cylinder_max).round()
            df_denorm['cylinders'] = df_denorm['cylinders'].map(self.cylinder_reverse)
        
        # Condition
        if 'condition' in df_denorm.columns:
            condition_max = max(self.condition_order.values())
            df_denorm['condition'] = (df_denorm['condition'] * condition_max).round()
            df_denorm['condition'] = df_denorm['condition'].map(self.condition_reverse)
        
        # Denormalize categorical columns
        for col in self.categorical_cols:
            if col in df_denorm.columns and col in self.categorical_mappings:
                max_val = self.categorical_mappings[col]['max']
                reverse_mapping = self.categorical_mappings[col]['reverse_mapping']
                df_denorm[col] = (df_denorm[col] * max_val).round()
                df_denorm[col] = df_denorm[col].map(reverse_mapping)
        
        # Handle encode-only columns
        for col in self.encode_only_cols:
            if col in df_denorm.columns and col in self.categorical_mappings:
                reverse_mapping = self.categorical_mappings[col]['reverse_mapping']
                df_denorm[col] = df_denorm[col].round()
                df_denorm[col] = df_denorm[col].map(reverse_mapping)
        
        return df_denorm



def analyze_feature_impact_by_model(folder_path, normalizer=None):
    """
    Analyzes feature impact on price for each car model separately,
    with data normalization and robust handling of missing values.
    
    Parameters:
    folder_path (str): Path to the folder containing vehicle data CSV files
    normalizer (DataNormalizer, optional): Pre-fitted normalizer instance
    
    Returns:
    dict: Dictionary containing correlation results by model
    """
    import pandas as pd
    import numpy as np
    from glob import glob
    import os
    import warnings
    
    # Suppress numpy warnings about division
    warnings.filterwarnings('ignore', category=RuntimeWarning)
    
    # Initialize or use provided normalizer
    if normalizer is None:
        normalizer = DataNormalizer()
        # Get all file paths for fitting
        file_paths = glob(os.path.join(folder_path, "cleaned-vehicles-*.csv"))
        if file_paths:
            print("Fitting normalizer on all data...")
            normalizer.fit(file_paths)
        else:
            print("No files found for fitting normalizer")
            return None
    
    # Columns to exclude from analysis
    exclude_columns = [
        'id', 'url', 'region_url', 'image_url', 'description', 
        'manufacturer', 'model', 'VIN', 'posting_date'
    ]
    
    def safe_correlation(x, y):
        """Calculate correlation with proper handling of edge cases"""
        # Remove any null values
        mask = ~(np.isnan(x) | np.isnan(y))
        x = x[mask]
        y = y[mask]
        
        # Need at least 3 points for meaningful correlation
        if len(x) < 3:
            return np.nan
            
        # Check for zero variance
        if np.std(x) == 0 or np.std(y) == 0:
            return np.nan
            
        try:
            return np.corrcoef(x, y)[0, 1]
        except:
            return np.nan
    
    # Initialize storage for model-specific correlations
    model_correlations = {}
    
    # Process each file
    file_paths = glob(os.path.join(folder_path, "cleaned-vehicles-*.csv"))
    
    if not file_paths:
        print("No files found matching the pattern 'cleaned-vehicles-*.csv'")
        return None
        
    # Process each file
    for file_path in file_paths:
        print(f"Processing {os.path.basename(file_path)}...")
        
        try:
            # Read the file
            df = pd.read_csv(file_path)
            
            # Process each model that has sufficient data
            for model in df['model'].unique():
                if pd.isna(model):
                    continue
                    
                # Get data for this model
                model_df = df[df['model'] == model].copy()
                
                # Require at least 50 samples for reliable correlation
                if len(model_df) < 50:
                    continue
                
                # Normalize the data
                try:
                    normalized_df = normalizer.normalize(model_df)
                except Exception as e:
                    print(f"Error normalizing data for model {model}: {str(e)}")
                    continue
                
                # Initialize correlations for this model if not exists
                if model not in model_correlations:
                    model_correlations[model] = {
                        'correlations': {},
                        'sample_size': 0,
                        'significant_correlations': {}
                    }
                
                # Update sample size
                model_correlations[model]['sample_size'] += len(normalized_df)
                
                # Calculate correlations for normalized features
                for column in normalized_df.columns:
                    if column in exclude_columns or column == 'price' or column == 'model':
                        continue
                    
                    # Handle numeric columns (all should be numeric after normalization)
                    if pd.api.types.is_numeric_dtype(normalized_df[column]):
                        correlation = safe_correlation(
                            normalized_df[column].astype(float), 
                            normalized_df['price'].astype(float)
                        )
                        
                        if not np.isnan(correlation):
                            # Update running average of correlations with weight by sample size
                            if column not in model_correlations[model]['correlations']:
                                model_correlations[model]['correlations'][column] = correlation
                            else:
                                prev_corr = model_correlations[model]['correlations'][column]
                                prev_samples = model_correlations[model]['sample_size'] - len(normalized_df)
                                new_corr = (prev_corr * prev_samples + correlation * len(normalized_df)) / model_correlations[model]['sample_size']
                                model_correlations[model]['correlations'][column] = new_corr
        
        except Exception as e:
            print(f"Error processing {file_path}: {str(e)}")
            continue
    
    # Filter for significant correlations and sort
    for model in model_correlations:
        # Consider correlations significant if abs value >= 0.1
        significant = {k: v for k, v in model_correlations[model]['correlations'].items() 
                      if abs(v) >= 0.1}
        model_correlations[model]['significant_correlations'] = dict(
            sorted(significant.items(), key=lambda x: abs(x[1]), reverse=True)
        )
    
    return model_correlations, normalizer

def visualize_model_correlations(model_correlations, min_samples=100, top_n_models=10):
    """
    Visualize the correlations for top models with sufficient samples.
    
    Parameters:
    model_correlations (dict): Output from analyze_feature_impact_by_model
    min_samples (int): Minimum number of samples required for visualization
    top_n_models (int): Number of top models to display
    """
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    # Filter models with sufficient samples
    valid_models = {model: data for model, data in model_correlations.items() 
                   if data['sample_size'] >= min_samples}
    
    if not valid_models:
        print(f"No models found with at least {min_samples} samples")
        return
    
    # Sort models by sample size and take top N
    top_models = dict(sorted(valid_models.items(), 
                           key=lambda x: x[1]['sample_size'], 
                           reverse=True)[:top_n_models])
    
    for model, data in top_models.items():
        if not data['significant_correlations']:
            continue
            
        # Create figure for this model
        plt.figure(figsize=(12, 6))
        
        # Get correlations and feature names
        correlations = data['significant_correlations']
        features = list(correlations.keys())
        values = list(correlations.values())
        
        # Create bar plot
        bars = plt.barh(range(len(features)), 
                       [abs(v) for v in values],
                       color=[plt.cm.RdBu(0.5 * (v + 1)) for v in values])
        
        # Customize plot
        plt.yticks(range(len(features)), features, fontsize=8)
        plt.xlabel('Absolute Correlation with Normalized Price')
        plt.title(f'Feature Impact for {model}\n(n={data["sample_size"]:,} samples)')
        
        # Add correlation values on bars
        for i, v in enumerate(values):
            plt.text(abs(v), i, f'{v:.3f}', 
                    va='center', fontsize=8,
                    color='black' if abs(v) < 0.5 else 'white')
        
        plt.tight_layout()
        plt.show()
        
        print(f"\nTop correlations for {model} (n={data['sample_size']:,}):")
        for feature, corr in data['significant_correlations'].items():
            print(f"{feature:30} : {corr:>8.3f}")
        print("\n" + "-"*50 + "\n")


def analyze_overall_feature_impact(folder_path, normalizer=None):
    """
    Analyzes feature impact on price across all cars, with proper normalization
    and robust handling of outliers.
    
    Parameters:
    folder_path (str): Path to the folder containing vehicle data CSV files
    normalizer (DataNormalizer, optional): Pre-fitted normalizer instance
    
    Returns:
    tuple: (overall_correlations, sample_sizes, normalizer)
    """
    import pandas as pd
    import numpy as np
    from glob import glob
    import os
    import warnings
    
    warnings.filterwarnings('ignore', category=RuntimeWarning)
    
    # Initialize or use provided normalizer
    if normalizer is None:
        normalizer = DataNormalizer()
        file_paths = glob(os.path.join(folder_path, "cleaned-vehicles-*.csv"))
        if file_paths:
            print("Fitting normalizer on all data...")
            normalizer.fit(file_paths)
        else:
            print("No files found for fitting normalizer")
            return None
    
    # Columns to exclude
    exclude_columns = [
        'id', 'url', 'region_url', 'image_url', 'description', 
        'model', 'VIN', 'posting_date', 'manufacturer'
    ]
    
    # Initialize storage for overall statistics
    correlations = {}
    sample_sizes = {}
    
    # Process each file
    file_paths = glob(os.path.join(folder_path, "cleaned-vehicles-*.csv"))
    
    if not file_paths:
        print("No files found matching the pattern 'cleaned-vehicles-*.csv'")
        return None
    
    for file_path in file_paths:
        print(f"Processing {os.path.basename(file_path)}...")
        
        try:
            # Read and normalize the data
            df = pd.read_csv(file_path)
            normalized_df = normalizer.normalize(df)
            
            # Calculate correlations for each feature
            for column in normalized_df.columns:
                if column in exclude_columns or column == 'price':
                    continue
                
                if pd.api.types.is_numeric_dtype(normalized_df[column]):
                    # Remove nulls and outliers
                    data = pd.DataFrame({
                        'feature': normalized_df[column],
                        'price': normalized_df['price']
                    }).dropna()
                    
                    # Remove outliers using IQR method
                    Q1 = data['feature'].quantile(0.25)
                    Q3 = data['feature'].quantile(0.75)
                    IQR = Q3 - Q1
                    mask = ~((data['feature'] < (Q1 - 1.5 * IQR)) | 
                            (data['feature'] > (Q3 + 1.5 * IQR)))
                    clean_data = data[mask]
                    
                    if len(clean_data) >= 50:  # Minimum sample size
                        correlation = clean_data['feature'].corr(clean_data['price'])
                        
                        if not np.isnan(correlation):
                            # Update weighted average correlation
                            if column not in correlations:
                                correlations[column] = correlation
                                sample_sizes[column] = len(clean_data)
                            else:
                                total_n = sample_sizes[column] + len(clean_data)
                                weight1 = sample_sizes[column] / total_n
                                weight2 = len(clean_data) / total_n
                                correlations[column] = (correlations[column] * weight1 + 
                                                      correlation * weight2)
                                sample_sizes[column] = total_n
        
        except Exception as e:
            print(f"Error processing {file_path}: {str(e)}")
            continue
    
    # Sort correlations by absolute value
    sorted_correlations = dict(sorted(correlations.items(), 
                                    key=lambda x: abs(x[1]), 
                                    reverse=True))
    
    return sorted_correlations, sample_sizes, normalizer

def visualize_overall_correlations(correlations, sample_sizes):
    """
    Visualize the overall feature correlations with price.
    
    Parameters:
    correlations (dict): Feature correlations with price
    sample_sizes (dict): Sample sizes for each feature
    """
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    # Create figure
    plt.figure(figsize=(15, 10))
    
    # Get features and correlation values
    features = list(correlations.keys())
    values = list(correlations.values())
    
    # Create color map based on correlation values
    colors = [plt.cm.RdBu(0.5 * (v + 1)) for v in values]
    
    # Create bar plot
    bars = plt.barh(range(len(features)), [abs(v) for v in values], color=colors)
    
    # Customize plot
    plt.yticks(range(len(features)), 
               [f"{f} (n={sample_sizes[f]:,})" for f in features], 
               fontsize=10)
    plt.xlabel('Absolute Correlation with Price')
    plt.title('Overall Feature Impact on Car Prices\n(Normalized Data)')
    
    # Add correlation values on bars
    for i, v in enumerate(values):
        plt.text(abs(v), i, f'{v:.3f}', 
                va='center', fontsize=10,
                color='black' if abs(v) < 0.5 else 'white')
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed results
    print("\nDetailed correlation results:")
    print("\nFeature                          Correlation    Sample Size")
    print("-" * 60)
    for feature, corr in correlations.items():
        print(f"{feature:30} : {corr:>8.3f}    n={sample_sizes[feature]:,}")

In [ ]:
#correlation_series, normalizer = analyze_feature_impact_across_files("data")

In [ ]:
#correlation_series, normalizer = analyze_feature_impact_across_files("data", normalizer, correlation_series)

In [ ]:
#model_correlations = analyze_feature_impact_by_model("data")
model_correlations, normalizer = model_correlations
visualize_overall__correlations(model_correlations, min_samples=100)